# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [185]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [186]:
# You may have to adjust the path
corpus = open('corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [187]:
pattern = 'Nils Holgersson'
width = 25

In [188]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [189]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [190]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [191]:
# Write your code here
len(set(words))

44256

Lowercased text

In [192]:
# Write your code here
len(set([w.lower() for w in words]))

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [193]:
# Write your code
nonletter = r"[^\p{L}.;:?!]+"

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [194]:
# Write your code here
def clean(text):
    return re.sub(nonletter, r' ', text)

In [195]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [196]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [197]:
# can a sentence end with ':;'?
sentence_boundaries = r"[.;:?!]+[ \n]+(\p{Lu})" # \r\n\t\f\v


##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [198]:
# Write your code here
sentence_markup = r' </s>\n<s> \1'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [199]:
# Write your code here
text = re.sub(sentence_boundaries, sentence_markup, test_para)


In [200]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [201]:
# Write your code here
text = '<s> ' + text + ' </s>'

In [202]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [203]:
# Write your code here
text = re.sub('[.]?[ ]+?', ' ', text)

In [204]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [205]:
# Write your code here
def segment_sentences(text):
    text = re.sub(sentence_boundaries, sentence_markup, text)
    text = '<s> ' + text + ' </s>'
    return re.sub('[.!?:;]+', '', re.sub('[ ]+', ' ', text.lower())) 
    

In [206]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

print(segment_sentences(clean(test_para)))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [207]:
# Write your code here
corpus = clean(corpus)
corpus = segment_sentences(corpus)

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [208]:
print(corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [209]:
# Write your code here
words = re.findall('[^ \n]+', corpus)

The five last lines of the corpus should like this:

In [210]:
print(len(set(words)))
print(words[-101:])

41031
['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [211]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [212]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [213]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [214]:
frequency_bigrams = bigrams(words)
print(len(list(frequency_bigrams)))
display(list(frequency_bigrams.items())[:20])

320122


[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [215]:
# Write your code
def unigram_lm(frequency, sent_words):
    print("=====================================================")
    print("wi\tC(wi)\t#words\t\tP(wi)")
    print("=====================================================")
    nbr_words = sum([x for w,x in frequency.items()])
    pm = 1
    for word in sent_words:
        count = frequency.get(word,0)
        pm *= count/nbr_words
        print(f"{word}\t{count}\t{nbr_words}\t\t{count/nbr_words}")
    print("=====================================================")
    print(f"Prob. unigrams: {pm}" )
    print(f"Geometric mean prob.: {pm**(1.0/len(sent_words))}")
    
    hl = -(1.0/len(sent_words)) * math.log(pm, 2)
    perplexity = 2**hl
    print(f"Entropy rate: {hl}")
    print(f"Perplexity: {perplexity}")
    return perplexity

In [216]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [217]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi	C(wi)	#words		P(wi)
det	21108	1041560		0.020265755213333847
var	12090	1041560		0.011607588617074388
en	13514	1041560		0.01297476861630631
gång	1332	1041560		0.001278850954337724
en	13514	1041560		0.01297476861630631
katt	16	1041560		1.5361573025077767e-05
som	16288	1041560		0.015638081339529167
hette	97	1041560		9.312953646453396e-05
nils	87	1041560		8.352855332386037e-05
</s>	59047	1041560		0.056690925150735434
Prob. unigrams: 5.3651155337425844e-27
Geometric mean prob.: 0.0023602494649885993
Entropy rate: 8.726844932328587
Perplexity: 423.68402782577465


In [218]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [219]:
# Write your code
def bigram_lm(frequency, frequency_bigrams, sent_words):
    print("=====================================================")
    print("wi\twi+1\tCi,i+1\tCi\tP(wi+1|wi)")
    print("=====================================================")

    nbr_words = sum([x for _,x in frequency.items()])
    pm = 1

    tupel_sent= []
    for i in range(len(sent_words)-1):
        tupel_sent.append((sent_words[i], sent_words[i+1]))

    for w1,w2 in tupel_sent:
        cw1 = frequency.get(w1,0)
        cw1w2 = frequency_bigrams.get((w1,w2),0)

        if cw1w2 == 0:
            p = cw1/nbr_words
            ps = f"0.0\t*backoff:\t{p}"
        else:
            p = cw1w2/cw1
            ps = str(p)

        pm *= p
        print(f"{w1}\t{w2}\t{cw1w2}\t{cw1}\t{ps}")

    print("=====================================================")
    print(f"Prob. unigrams: {pm}" )
    print(f"Geometric mean prob.: {pm**(1.0/(len(sent_words)-1))}")
    
    if pm > 0:
        hl = -(1.0/(len(sent_words)-1)) * math.log(pm, 2)
    else: 
        hl = 0
    perplexity = 2**hl
    print(f"Entropy rate: {hl}")
    print(f"Perplexity: {perplexity}")
    return perplexity

In [220]:
sentence = '<s> det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	det	5672	59047	0.09605907158704083
det	var	3839	21108	0.1818741709304529
var	en	712	12090	0.058891645988420185
en	gång	706	13514	0.052242119283705785
gång	en	20	1332	0.015015015015015015
en	katt	6	13514	0.0004439840165754033
katt	som	2	16	0.125
som	hette	45	16288	0.002762770137524558
hette	nils	0	97	0.0	*backoff:	9.312953646453396e-05
nils	</s>	2	87	0.022988505747126436
Prob. unigrams: 2.6492927307096436e-19
Geometric mean prob.: 0.013877556200403469
Entropy rate: 6.171102654133041
Perplexity: 72.05879663243059


In [221]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

In [222]:
sentence = '<s> hej jag är en liten mört som har simmat bort </s>'
sent_words = sentence.split()
bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	hej	1	59047	1.6935661422256845e-05
hej	jag	0	3	0.0	*backoff:	2.8802949422020816e-06
jag	är	404	9511	0.04247713174219325
är	en	304	6290	0.04833068362480127
en	liten	419	13514	0.03100488382418233
liten	mört	0	592	0.0	*backoff:	0.0005683782019278774
mört	som	0	0	0.0	*backoff:	0.0
som	har	372	16288	0.022838899803536344
har	simmat	0	4666	0.0	*backoff:	0.004479818733438304
simmat	bort	0	0	0.0	*backoff:	0.0
bort	</s>	75	1058	0.07088846880907372
Prob. unigrams: 0.0
Geometric mean prob.: 0.0
Entropy rate: 0
Perplexity: 1


1

In [223]:
sentence = '<s> den som väntar på något gott väntar aldrig för länge </s>'
sent_words = sentence.split()
bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	den	1375	59047	0.023286534455603164
den	som	434	11624	0.03733654507914659
som	väntar	4	16288	0.0002455795677799607
väntar	på	17	75	0.22666666666666666
på	något	113	14250	0.007929824561403509
något	gott	14	1996	0.0070140280561122245
gott	väntar	0	417	0.0	*backoff:	0.0004003609969660893
väntar	aldrig	0	75	0.0	*backoff:	7.200737355505203e-05
aldrig	för	1	1420	0.0007042253521126761
för	länge	27	9443	0.0028592608281266547
länge	</s>	37	555	0.06666666666666667
Prob. unigrams: 1.0417244539874559e-26
Geometric mean prob.: 0.0043448774624093075
Entropy rate: 7.846468795048716
Perplexity: 230.15608809494088


230.15608809494088

In [224]:
sentence = '<s> man ska inte ropa hej förrän man kommit över bäcken </s>'
sent_words = sentence.split()
bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	man	280	59047	0.004741985198231917
man	ska	12	2322	0.00516795865633075
ska	inte	110	1886	0.05832449628844114
inte	ropa	2	11355	0.00017613386173491853
ropa	hej	0	87	0.0	*backoff:	8.352855332386037e-05
hej	förrän	0	3	0.0	*backoff:	2.8802949422020816e-06
förrän	man	3	355	0.008450704225352112
man	kommit	2	2322	0.0008613264427217916
kommit	över	21	879	0.023890784982935155
över	bäcken	2	2454	0.0008149959250203749
bäcken	</s>	4	17	0.23529411764705882
Prob. unigrams: 2.019773216263824e-30
Geometric mean prob.: 0.0019974782327573212
Entropy rate: 8.967604502992776
Perplexity: 500.63123772798184


500.63123772798184

In [225]:
sentence = '<s> som man bäddar får man ligga </s>'
sent_words = sentence.split()
bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	som	43	59047	0.0007282334411570444
som	man	127	16288	0.007797151277013753
man	bäddar	0	2322	0.0	*backoff:	0.002229348285264411
bäddar	får	0	6	0.0	*backoff:	5.760589884404163e-06
får	man	15	906	0.016556291390728478
man	ligga	0	2322	0.0	*backoff:	0.002229348285264411
ligga	</s>	4	232	0.017241379310344827
Prob. unigrams: 4.640496492852036e-20
Geometric mean prob.: 0.0017301375643702093
Entropy rate: 9.174897532612507
Perplexity: 577.9887221649989


577.9887221649989

In [226]:
sentence = '<s> så går vi snart </s>'
sent_words = sentence.split()
bigram_lm(frequency, frequency_bigrams, sent_words)

wi	wi+1	Ci,i+1	Ci	P(wi+1|wi)
<s>	så	657	59047	0.011126729554422747
så	går	19	9149	0.0020767296972346704
går	vi	3	633	0.004739336492890996
vi	snart	6	2105	0.0028503562945368173
snart	</s>	13	752	0.017287234042553192
Prob. unigrams: 5.3962208075959695e-12
Geometric mean prob.: 0.005577222136401242
Entropy rate: 7.486237551122617
Perplexity: 179.3007299230976


179.3007299230976

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [227]:
# Write your code
def trigrams(words):
    trigrams = []
    for i in range(len(words)-2):
        trigrams.append((words[i], words[i+1], words[i+2]))
    frequency_trigrams = {}
    for i in range(len(words)-2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else: 
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [228]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

In [229]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [230]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [231]:
cand_nbr = 5

In [232]:
# Write your code here
picks = []
for (w1,w2),val in frequency_bigrams.items():
    if w1 == '<s>':
        if starting_text in w2[:len(starting_text)]:
            picks.append((val, w2)) 
picks = sorted(picks, key=lambda x: x[1])
picks = sorted(picks, key=lambda x: x[0], reverse=True)
current_word_predictions_1 = [t[1] for t in picks[:cand_nbr]]

In [233]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [234]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [235]:
# Write your code here
tokens=current_text.strip().split()

In [236]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [237]:
# Write your code here
m1,m2 = tokens[1], tokens[2]
picks = []
for (w1,w2,w3), val in frequency_trigrams.items():
    if w1 == m1 and w2 == m2:
        picks.append((val, w3))
picks = sorted(picks, key=lambda x: x[1])
picks = sorted(picks, key=lambda x: x[0], reverse=True)
next_word_predictions = [t[1] for t in picks[:cand_nbr]]

In [238]:
next_word_predictions

['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [239]:
current_text = "Det var en g".lower()

In [240]:
# Write your code here
tokens = current_text.split()
m1,m2 = tokens[1], tokens[2]
picks = []
for (w1,w2,w3), val in frequency_trigrams.items():
    if w1 == m1 and w2 == m2 and tokens[3] in w3[:len(tokens[3])]:
        picks.append((val, w3))
# picks = sorted(picks, key=lambda x: (x[0], x[1]))
picks = sorted(picks, key=lambda x: x[1])
picks = sorted(picks, key=lambda x: x[0], reverse=True)
current_word_predictions_2 = [t[1] for t in picks[:cand_nbr]]
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

In [241]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [242]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

In [243]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [244]:
STIL_ID = ["jo4383ba-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [245]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "detta", "denna"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [246]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [247]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [248]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '5edc648eabab0cdc7e2027be37348ab07ed61f8bd450da187d0f1107c16c2c836a728c017371163468422a0b1c7b276363c0e1223fa705aacb0ca44eb2d23e08',
 'submission_id': '3342b150-4f2c-42a6-957c-5a84cdd2dfb2'}

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to include the unigram and bigram tables for five sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 24, 2021.